In [1]:
import os
import time
# import ants
import itk
# from itkwidgets import view --> This seems to cause the issue when an error happened, the kernel disconnects and stays in a zombie state
# import SimpleITK as sitk  # An alternative to itk
import glob
import shutil
import numpy as np
# import pydicom as pyd
# import dicom2nifti
from directory_tree import display_tree  # Nice tool to display directory trees (https://pypi.org/project/directory-tree/)

from datetime import timedelta

In [2]:
def list_folder_content(path, show_hidden=False):
    if show_hidden:
        ddfldrlst = os.listdir(path)
    else:
        ddfldrlst = list(filter(lambda item: not item.startswith('.'),os.listdir(path)))      
    return ddfldrlst

def display_folder_list(file_list):
    print('\n'.join(f'[{idx}] - {file_idx}' for idx, file_idx in enumerate(file_list)))

def get_path_to_process(full_path):
    print('Folder content:')
    print(display_tree(full_path, header=True, string_rep=True, show_hidden=False, max_depth=2))
    folder_content = list_folder_content(full_path)
    # Ideally we'll have only one sub-folder inside the PreTreatment folder. If more than one, then we have to choose, but by default, we'll select the first one.
    idx_reg = 0
    if len(folder_content) > 1:
        display_folder_list(folder_content)
        idx_sel = input(f'Select the folder with the dataset_to_process to process (0-{len(folder_content)-1} or just press Enter to proceed with sub-folder {folder_content[idx_reg]}):')
        if idx_sel:
            idx_reg = int(idx_sel)
    path2data = os.path.join(full_path, folder_content[idx_reg])
    print(f'Will process {folder_content[idx_reg]}')
    return path2data

def check_time_points(path_to_check, nmax = 6):
    if path_to_check is not None:
        nr_of_folders = list_folder_content(path_to_check)
        print('Folder seems Ok:' if len(nr_of_folders)== nmax else f'Error! Check path {path_to_check} is the correct one:')
        display_tree(path_to_check, max_depth=1)
        return nr_of_folders if len(nr_of_folders) == nmax else None
    else:
        return None

def add_prefix_to_filename(full_path, prefix=None):
    # Assume the last part of the path is the filename (with extension)
    file_path, file_name_ext = os.path.split(full_path)
    if prefix:
        updated_filename = '_'.join([prefix, file_name_ext])
        return os.path.join(file_path, updated_filename)
    else:
        return prefix

In [3]:
HOMEPATH = os.getenv('HOME')
SRCPATH = os.path.join(HOMEPATH, 'Data', 'fMRIBreastData')

DCMSRCFLDR = 'StudyData'
NIFTISRCFLDR = 'NiftiData'
OUTPUTFLDR = 'ElastixReg'
CONFIGFLDR = 'configFiles'

studypath = os.path.join(SRCPATH, NIFTISRCFLDR)
savepath = os.path.join(SRCPATH, OUTPUTFLDR)
cfgpath = os.path.join(SRCPATH, CONFIGFLDR)
# Check whether the folder SAVEPATH exists or not, if not, attempts to create it
os.makedirs(savepath, exist_ok=True)

In [4]:
DEBUGMODE = False

In [5]:
patients = list_folder_content(studypath)
print('Patient data folders:')
display_folder_list(patients)
idx_sel = int(input(f'Select a patient index (0-{len(patients)-1})'))
if idx_sel > len(patients)-1:
    print(f'ERROR!: Selected index {idx_sel} is out of range for the PATIENTS list (it must be ≤{len(patients)-1})')

Patient data folders:
[0] - ANON18218
[1] - ANON89073
[2] - ANON98269
[3] - ANON68760
[4] - ANON97378
[5] - RICE001
[6] - ANON99397


Select a patient index (0-6) 6


In [6]:
patient = patients[idx_sel]
data_patient = os.path.join(studypath, patient)
print(display_tree(data_patient, header=True, string_rep=True, show_hidden=False, max_depth=4))

ANON99397/
└── ANON18218/
    └── 20230720/
        └── 301/
            ├── 1/
            ├── 2/
            ├── 3/
            ├── 4/
            ├── 5/
            └── 6/



In [7]:
prefix_folder = input('If there is a sub-folder between the PATIENT and VISITS, type it here (if there is no, leave it blank and press ENTER):')
patient_path = os.path.join(data_patient, prefix_folder)
visits = list_folder_content(patient_path)
print(f'There are {len(visits)} visit{"s" if len(visits)>1 else ""} from patient {patient}:')
display_folder_list(visits)
print('The earliest visit is the pre-treatment and the latest, the post-treatment. If there is only one visit, we assume it is the pre-treatment')

If there is a sub-folder between the PATIENT and VISITS, type it here (if there is no, leave it blank and press ENTER): ANON18218


There are 1 visit from patient ANON99397:
[0] - 20230720
The earliest visit is the pre-treatment and the latest, the post-treatment. If there is only one visit, we assume it is the pre-treatment


In [8]:
visit_dates = [int(visit) for visit in visits]
indices = [ elem[0] for elem in sorted( enumerate(visit_dates), key = lambda pair : pair[1] )]
data_visits = {'PreTreatment':''}
if len(visits) > 1:
    data_visits['PostTreatment'] = ''
    
for idx, idvisit in enumerate(data_visits):
    data_visits[idvisit] = visits[indices[idx]]
if len(data_visits) < 2:
    data_visits['PostTreatment'] = None
print('\n'.join([f'{visit} Folder: {date}' for visit, date in data_visits.items()]))

PreTreatment Folder: 20230720
PostTreatment Folder: None


In [9]:
# Pre-Treatment Registration
print('Select Pre-treatment dataset_to_process:')
pre_treatment_path = os.path.join(patient_path, data_visits['PreTreatment'])
pre_treat_data_path = get_path_to_process(pre_treatment_path)

# Post-Treatment Registration (only if there is data available)
print(''.join(['§']*100))
print('Select Post-treatment dataset_to_process:')
if data_visits['PostTreatment'] != None:
    post_treatment_path = os.path.join(patient_path, data_visits['PostTreatment'])
    post_treat_data_path = get_path_to_process(post_treatment_path)
else:
    post_treatment_path = None
    post_treat_data_path = None
    print('Nothing to process')

Select Pre-treatment dataset_to_process:
Folder content:
20230720/
└── 301/
    ├── 1/
    ├── 2/
    ├── 3/
    ├── 4/
    ├── 5/
    └── 6/

Will process 301
§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§
Select Post-treatment dataset_to_process:
Nothing to process


We're interested only in the DCEMRI data. There should be 6 sub-folder labelled 1-6, where each one represent a 3D volume on a timepoint:
* 1: pre-contrast
* 2-6: post-contrast


In [10]:
# Check the selected folders contains the 6 timepoints
visit_desc = {'PreTreatment': {'timepoints': check_time_points(pre_treat_data_path),
                              'datapath': pre_treat_data_path},
             'PostTreatment': {'timepoints': check_time_points(post_treat_data_path),
                               'datapath': post_treat_data_path}
             }

Folder seems Ok:
301/
├── 1/
├── 2/
├── 3/
├── 4/
├── 5/
└── 6/


In [11]:
# Load the data (Nifti format) using ANTs
print(f'Processing dataset_to_process {patient_path}')
dataset_to_process = {'PatientID': patient,
           'PreTreatment': {},
           'PostTreatment': {}
          }
# By default, we consider the first timepoint (i.e pre-contrast) to be the Fixed image (or reference Space), but it can be changed here, by setting the index to any other timepoint
fixed_volume_pos = 1  # It is a position, not an index, that's why start from 1 instead of 0

# All other images in the timeseries will be labelled as "moving"    
for visit_name, description in visit_desc.items():
    if description['datapath'] is not None:
        print(f'Loading images from {visit_name} folder...')
        for idx_data in description['timepoints']:
            dataset_to_process[visit_name][idx_data] = {}
            nii_filepath = os.path.join(description['datapath'], idx_data)
            nii_files = list_folder_content(nii_filepath)
            if len(nii_files) > 1:
                print(f'WARNING!: Folder {nii_filepath} seems to have more than one volume:')
                display_folder_list(nii_files)
                break
            print(f'TimePoint {idx_data}, Datafile: {nii_files[0]}')
            print(f'Loading image volume ...')
            dataset_to_process[visit_name][idx_data]['path'] = os.path.join(description['datapath'], idx_data, nii_files[0])
            # dataset_to_process[visit_name][idx_data]['img_data'] = ants.image_read(dataset_to_process[visit_name][idx_data]['path'])
            dataset_to_process[visit_name][idx_data]['img_data'] = itk.imread(dataset_to_process[visit_name][idx_data]['path'])
            dataset_to_process[visit_name][idx_data]['time_point'] = int(idx_data)
            if idx_data == '1':
                dataset_to_process[visit_name][idx_data]['DCE_ref'] = 'Pre-Contrast'
            else:
                dataset_to_process[visit_name][idx_data]['DCE_ref'] = 'Post-Contrast'

            if int(idx_data) == fixed_volume_pos:
                dataset_to_process[visit_name][idx_data]['reg_ref'] = 'Fixed'
            else:
                dataset_to_process[visit_name][idx_data]['reg_ref'] = 'Moving'
        print(''.join(['§']*100))
        if DEBUGMODE:
            print(dataset_to_process)
print('Finished loading the data')

Processing dataset_to_process /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218
Loading images from PreTreatment folder...
TimePoint 6, Datafile: 301_dyn_ethrive.nii.gz
Loading image volume ...
TimePoint 1, Datafile: 301_dyn_ethrive.nii.gz
Loading image volume ...
TimePoint 4, Datafile: 301_dyn_ethrive.nii.gz
Loading image volume ...
TimePoint 3, Datafile: 301_dyn_ethrive.nii.gz
Loading image volume ...
TimePoint 2, Datafile: 301_dyn_ethrive.nii.gz
Loading image volume ...
TimePoint 5, Datafile: 301_dyn_ethrive.nii.gz
Loading image volume ...
§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§
Finished loading the data


In [12]:
# Registration parameters (this is the meat of the work!)

# ANTs
# For details about possible values and description of parameters, see the help page: https://antspy.readthedocs.io/en/latest/registration.html
# Default values (as listed in the hep page)
par_set = {'type_of_transform': 'SyN', 
               'initial_transform': None, 
               'outprefix': '', 
               'mask': None, 
               'moving_mask': None, 
               'mask_all_stages': False, 
               'grad_step': 0.2, 
               'flow_sigma': 3, 
               'total_sigma': 0, 
               'aff_metric': 'mattes', 
               'aff_sampling': 32, 
               'aff_random_sampling_rate': 0.2, 
               'syn_metric': 'mattes', 
               'syn_sampling': 32, 
               'reg_iterations': (40, 20, 0), 
               'aff_iterations': (2100, 1200, 1200, 10), 
               'aff_shrink_factors': (6, 4, 2, 1), 
               'aff_smoothing_sigmas': (3, 2, 1, 0), 
               'write_composite_transform': False, 
               'random_seed': None}

# To ensure reproducibility of the results, set the random_seed to a constant value:
par_set['random_seed'] = 42 #(just to keep along with the pop-culture reference, e.g. https://medium.com/geekculture/the-story-behind-random-seed-42-in-machine-learning-b838c4ac290a


# Elastix
if DEBUGMODE:
    print(f'Define the parameters for the registration')
par_set = itk.ParameterObject.New()
par_set.AddParameterFile(os.path.join(cfgpath, 'Par0032_rigid.txt'))
par_set.AddParameterFile(os.path.join(cfgpath, 'Par0032_bsplines.txt'))
if DEBUGMODE:
    print('Parameters for ELASTIX:')
    print(par_set)
# parameter_object.AddParameterMap(default_rigid_parameter_map)

In [13]:
moving_dataset = {'PreTreatment': {},
             'PostTreatment': {}
            }
fixed_dataset = {'PreTreatment': {},
             'PostTreatment': {}
            }
for visit_name, description in visit_desc.items():
    if description['datapath'] is not None:
        print(f'Selecting FIXED and MOVING dataset_to_processs from patient {patient} - {visit_name} visit...')
        for timepoint, dataset_to_process_i in dataset_to_process[visit_name].items():
            if (dataset_to_process_i['reg_ref']=='Fixed'):
                print(f'Fixed image is: {dataset_to_process_i["path"]}')
                fixed_volume = dataset_to_process_i['img_data']
                fixed_vol_source_path = dataset_to_process_i['path']
                path_to_fixed_output_vol = add_prefix_to_filename(fixed_vol_source_path.replace(NIFTISRCFLDR,OUTPUTFLDR), prefix=f'TP{fixed_volume_pos:02d}_FIXED')
                if DEBUGMODE:
                    print(f'Fixed Image will saved as: {path_to_fixed_output_vol}')
                    print(f'Create the output path: {os.path.split(path_to_fixed_output_vol)[0]}')
                os.makedirs(os.path.split(path_to_fixed_output_vol)[0], exist_ok=True)
                shutil.copy2(fixed_vol_source_path, path_to_fixed_output_vol)
                fixed_dataset[visit_name] = {'fixed_volume': fixed_volume,
                                       'fixed_vol_source_path': fixed_vol_source_path,
                                       'path_to_fixed_output_vol': path_to_fixed_output_vol}
            elif (dataset_to_process_i['reg_ref'] == 'Moving'):
                print(f'Moving image is: {dataset_to_process_i["path"]}')
                moving_dataset[visit_name][dataset_to_process_i['time_point']] = {'moving_volume': dataset_to_process_i['img_data'],
                                                   'moving_vol_source_path': dataset_to_process_i['path']}
                path_to_moving_output_vol = add_prefix_to_filename(dataset_to_process_i['path'].replace(NIFTISRCFLDR,OUTPUTFLDR), prefix=f"TP{dataset_to_process_i['time_point']:02d}_MOVED_WRT_TPOINT{fixed_volume_pos:02d}")
                moving_dataset[visit_name][dataset_to_process_i['time_point']]['path_to_moving_output_vols'] = path_to_moving_output_vol
                if DEBUGMODE:
                    print(f'Moving Image will saved as: {path_to_moving_output_vol}')
                    print(f'Create the output path: {os.path.split(path_to_moving_output_vol)[0]}')
                os.makedirs(os.path.split(path_to_moving_output_vol)[0], exist_ok=True)
        print(''.join(['§']*100))
print('Finished preparing the data for running the registration process')

Selecting FIXED and MOVING dataset_to_processs from patient ANON99397 - PreTreatment visit...
Moving image is: /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/6/301_dyn_ethrive.nii.gz
Fixed image is: /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/1/301_dyn_ethrive.nii.gz
Moving image is: /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/4/301_dyn_ethrive.nii.gz
Moving image is: /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/3/301_dyn_ethrive.nii.gz
Moving image is: /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/2/301_dyn_ethrive.nii.gz
Moving image is: /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/5/301_dyn_ethrive.nii.gz
§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§
Finished preparing the data for running the registration process


In [14]:
registration_output = {}
init_time = time.perf_counter()
for visit_name, description in visit_desc.items():
    if description['datapath'] is not None:
        print(f'Registering {visit_name} dataset_to_process, please wait...')
        # Initialise the output by adding the fixed image into the dictionary so the whole set can be concatenated into a single 4D dataset_to_process:
        fixed_volume = fixed_dataset[visit_name]['fixed_volume']
        registration_output[visit_name] = {fixed_volume_pos: {'warpedmovout': fixed_volume, 
                                                            'warpedfixout': fixed_volume,
                                                            'fwdtransforms': None,
                                                            'invtransforms': None}
                                        }
        nt = len(moving_dataset[visit_name])+1

        # Initialise the image dimensions for the concatenation
        # Will use the parameters from the corresponding Fixed image, adding the extra (4th) dimension for the time points:
#         spacing = fixed_volume.spacing + (1,)
#         origin = fixed_volume.origin + (0,)
#         volume_size = fixed_volume.shape + (nt,)
#         directions = np.eye(4)
#         directions[:-1, :-1] = fixed_volume.direction

#         # At the end of the loop, concatenate the unregistered and registered images into single 4D multiarrays:
#         template_4d = ants.make_image(imagesize=volume_size,
#                                       spacing=spacing, 
#                                       origin=origin, 
#                                       direction=directions)
        output_volume_path = description['datapath'].replace(NIFTISRCFLDR, OUTPUTFLDR)

        # ANTs concatenation output:
        registered_series = [None]*nt
        unregistered_series = [None]*nt
        
        # Initialise the concatenation with the fixed (i.e. ref) volume:
        registered_series[fixed_volume_pos-1] = fixed_volume
        unregistered_series[fixed_volume_pos-1] = fixed_volume
        
        
        # ITK concatenation output: 
        # To stack the volumes, use the function TileFilter, following the example at 
        # https://examples.itk.org/src/filtering/imagegrid/create3dvolume/documentation 
        # However, as I found out the hard way, the method SetInput works in order, even if it is within a loop, it works lexicographically (i.e. ordinal numbers)
        # so we'll recycle the list required for ANTs and after the registration, populates the tile in another (much quicker) loop
        input_dimension = fixed_volume.GetImageDimension()
        pixel_type = itk.template(fixed_volume)[1][0]
        output_dimension = input_dimension + 1

        input_image_type = itk.Image[pixel_type, input_dimension]
        output_image_type = itk.Image[pixel_type, output_dimension]
        
        layout = [1, 1, 1, nt]
        registered_tiles = itk.TileImageFilter[input_image_type, output_image_type].New()
        registered_tiles.SetLayout(layout)

        unregistered_tiles = itk.TileImageFilter[input_image_type, output_image_type].New()
        unregistered_tiles.SetLayout(layout)

        # Initialise the concatenation with the fixed (i.e. ref) volume:
        # TODO: might not be necessary a default value
        # defaultPixelValue = 128
        # unregistered_tiles.SetDefaultPixelValue(defaultPixelValue)
        if DEBUGMODE:
            print(f'Fixed Volume ID: {fixed_volume_pos}')
            print(f'Fixed Volume index: {fixed_volume_pos-1}')
        
        # Start the timer to assess computation time of the registration per volume
        init_time_pv = time.perf_counter()
        
        for idx_set, moving_dataset_i in moving_dataset[visit_name].items():
            if DEBUGMODE:
                print(f'Index: {idx_set}')
            # Initialise a time variable to measure the elapsed time taken during registration:
            moving = moving_dataset_i['moving_volume']
            
            start_time = time.perf_counter()
            print(f"Registering moving data at {moving_dataset_i['moving_vol_source_path']} to fixed image at {fixed_dataset[visit_name]['fixed_vol_source_path']}...")

# RUNNING THE REGISTRATION:
            # ANTS:
            # registeredOutput = ants.registration(fixed=fixed_volume , moving=moving, **par_set) #type_of_transform='SyN')
            # ELASTIX:
            warped_moving, result_transform_pars = itk.elastix_registration_method(fixed_volume , 
                                                                                moving, 
                                                                                parameter_object=par_set,
                                                                                log_to_console=False) 
# END OF REGISTRATION

            end_time = time.perf_counter()
            print(f"Finished registration of {moving_dataset_i['moving_vol_source_path']}")
            elp_time = end_time - start_time
            print(f'Elapsed Time: {elp_time:0.2f}[s] ({timedelta(seconds=elp_time)})')
            
# SORTING OUT THE RESULTS:
            # ANTS:
            # warped_moving = registeredOutput['warpedmovout']
            # ELASTIX:
            registration_output[visit_name][idx_set] = {'warpedmovout': warped_moving,
                                                        'fwdtransforms': result_transform_pars}
            if DEBUGMODE:
                print(result_transform_pars)
            print(f"Saving output in {moving_dataset_i['path_to_moving_output_vols']}")
# END OF SORTING OUT

# WRITING OUT THE INDIVIDUAL RESULT:
            # ANTs:
            # ants.image_write( warped_moving, moving_dataset_i['path_to_moving_output_vols'])
            # ELASTIX:
            itk.imwrite(warped_moving, moving_dataset_i['path_to_moving_output_vols'])

# CONCATENATING THE TIME SERIES:
            # ANTs:
            # Registered Dataset (comment out if not run the actual registration):
            registered_series[idx_set-1] = registration_output[visit_name][idx_set]['warpedmovout']
            # Unregistered (original raw) Dataset:
            unregistered_series[idx_set-1] = moving
            
            # ITK Concatenation:
            # TODO: remove this
            # Registered Dataset:
            # registered_series.SetInput(idx-1, registration_output[visit_name][idx_set]['warpedmovout'])
            # Unregistered Dataset:
            # unregistered_series.SetInput(idx-1, moving)

            print(''.join(['§']*100))

        print(f'Concatenating datasets and saving 4D volumes at {output_volume_path}. Please wait...')
        registered_output_path = os.path.join(output_volume_path, 'RegisteredVolumes.nii.gz')
        unregistered_output_path = os.path.join(output_volume_path, 'UnregisteredVolumes.nii.gz')

        # ITK writing out concatenation results:
        # unregistered_series.Update()
        # SetInput must be in lexicographical order, so cannot be populated directly in the registration loop
        for idx, (reg_vol_i, ureg_vol_i) in enumerate(zip(registered_series, unregistered_series)):
            registered_tiles.SetInput(idx, reg_vol_i)
            unregistered_tiles.SetInput(idx, ureg_vol_i)

        # Registered Dataset:
        reg_writer = itk.ImageFileWriter[output_image_type].New()
        reg_writer.SetFileName(registered_output_path)
        reg_writer.SetInput(registered_tiles.GetOutput())
        reg_writer.Update()

        # Unregistered Dataset:
        ureg_writer = itk.ImageFileWriter[output_image_type].New()
        ureg_writer.SetFileName(unregistered_output_path)
        ureg_writer.SetInput(unregistered_tiles.GetOutput())
        ureg_writer.Update()
        
        
        # ANTs Concatenation:
        # registered_4d_series = ants.list_to_ndimage(template_4d, registered_series)
        # unregistered_4d_series = ants.list_to_ndimage(template_4d, unregistered_series)
        # ANTs writing out concatenation results:
        # ants.image_write(registered_4d_series, registered_output_path)
        # ants.image_write(unregistered_4d_series, unregistered_output_path)

        final_time_pv = time.perf_counter()
        elp_per_vol = final_time_pv - init_time_pv
        print(f'Total elapsed time per volume(including saving the data): {elp_per_vol:0.2f}[s] ({timedelta(seconds=elp_per_vol)})')
        print(''.join(['§']*100))

final_time = time.perf_counter()
elp_global = final_time - init_time
print(f'Finished processing datasets in {patient_path}')
print(f'Total elapsed time per patient (including saving the data): {elp_global:0.2f}[s] ({timedelta(seconds=elp_global)})')

Registering PreTreatment dataset_to_process, please wait...
Registering moving data at /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/6/301_dyn_ethrive.nii.gz to fixed image at /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/1/301_dyn_ethrive.nii.gz...
Finished registration of /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/6/301_dyn_ethrive.nii.gz
Elapsed Time: 35.87[s] (0:00:35.866261)
Saving output in /Users/joseulloa/Data/fMRIBreastData/ElastixReg/ANON99397/ANON18218/20230720/301/6/TP06_MOVED_WRT_TPOINT01_301_dyn_ethrive.nii.gz
§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§
Registering moving data at /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/4/301_dyn_ethrive.nii.gz to fixed image at /Users/joseulloa/Data/fMRIBreastData/NiftiData/ANON99397/ANON18218/20230720/301/1/301_dyn_ethrive.nii.gz...
Finish

In [34]:
# Run single instances to test parameters
vname = 'PreTreatment'
idx = 4
fix_set = fixed_dataset[vname]
mov_set = moving_dataset[vname][idx]

if DEBUGMODE:
    print(f'Started registration process, please wait...')
    
reg_result, result_transform_parameters = itk.elastix_registration_method(
    fix_set['fixed_volume'], mov_set['moving_volume'],
    parameter_object=parameter_object,
    log_to_console=False)


# 1) Basic Elastix Registration
test_name = 'BasicElastixRegistration'
if DEBUGMODE:
    print(f'Finished the registration run. Check the results ;) ')
itk.imwrite(reg_result, os.path.join(os.path.split(mov_set['path_to_moving_output_vols'])[0],
                                     f'Test_{test_name}_concat_volume.nii.gz'))

# par_set['random_seed'] = 42
# reg_result = ants.registration(fixed=fix_set['fixed_volume'] , moving=mov_set['moving_volume'], **par_set) #type_of_transform='SyN')

# ants.image_write( reg_result['warpedmovout'], mov_set['path_to_moving_output_vols'])
# reg_cat_vol = ants.list_to_ndimage(concat_vol, [fix_set['fixed_volume'], reg_result['warpedmovout']])

# ants.image_write(reg_cat_vol, 
#                  os.path.join(os.path.split(mov_set['path_to_moving_output_vols'])[0],
#                               f'Test_{test_name}_concat_volume.nii.gz'))


Started registration process, please wait...
Finished the registration run. Check the results ;) 


[<itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x1412bb390> >,
 <itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x157cf3b70> >,
 <itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x1412bbe40> >,
 <itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x1412bbc60> >,
 <itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x141249750> >,
 <itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x157cf3d50> >]

In [ ]:
itk.CastImageFilter(registered_series[0], itk.Image.UC3].New())

In [ ]:
1
